# Strategy Analysis

This notebook provides the analysis of the various strategies created in the `mancala.strategy` script. We will look into:
- running simulations to see which player strategy wins,
- potentially running some simple statistical tests to check whether there is a meaningful difference, and
- trying to look into the best moves across different strategies and simulations, e.g. is it always best to choose the goal making move if you are the starting player.

In [7]:
import json
from collections import Counter
from pprint import pprint
from typing import Final

from mancala.mancala import Player
from mancala.simulation import SimulationLoop
from mancala.strategy import (
    AlwaysMaximumPlayerStrategy,
    AlwaysMinimumPlayerStrategy,
    EvenGoalOrPiecesOnOtherSideStrategy,
    EvenGoalStealAndPiecesOnOtherSideStrategy,
    ExampleRandomPlayerStrategy,
)

In [21]:
def run_simulation(loop: SimulationLoop, simulations: int = 5000) -> Counter:
    winning_players = []
    for _ in range(simulations):
        loop.run(reset_simulation=True)
        winning_players.append(loop.winning_player)
    return Counter(winning_players)

def pprint_simulation_results(loop: SimulationLoop) -> None:
    pprint(json.loads(loop.serialize()))

## Random v.s. Random

With two random player strategies, we would expect that, except for ties, each player should win about the same amount of games. One curious thing to look into is whether there is a starting player advantage, i.e. does the starting player come with an advantage of winning more games?

### With Player.ONE starting

In [27]:
rand_vs_rand_p1 = SimulationLoop(
    player_one=ExampleRandomPlayerStrategy(),
    player_two=ExampleRandomPlayerStrategy(),
    starting_player=Player.ONE
)
pprint_simulation_results(rand_vs_rand_p1)

{'boards': [{'one': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0},
             'two': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0}}],
 'player_strategies': {'one': 'random-selection', 'two': 'random-selection'},
 'starting_player': 'one',
 'turns': [],
 'winning_player': None}


In [28]:
rand_vs_rand_p1.run()
pprint_simulation_results(rand_vs_rand_p1)

{'boards': [{'one': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0},
             'two': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0}},
            {'one': {'bins': [4, 5, 5, 5, 5, 0], 'goal': 0},
             'two': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0}},
            {'one': {'bins': [4, 5, 5, 5, 5, 0], 'goal': 0},
             'two': {'bins': [4, 5, 5, 5, 5, 0], 'goal': 0}},
            {'one': {'bins': [5, 6, 0, 5, 5, 0], 'goal': 1},
             'two': {'bins': [4, 5, 5, 5, 6, 1], 'goal': 0}},
            {'one': {'bins': [5, 6, 0, 5, 6, 1], 'goal': 1},
             'two': {'bins': [5, 6, 0, 5, 6, 1], 'goal': 1}},
            {'one': {'bins': [6, 0, 0, 5, 6, 1], 'goal': 2},
             'two': {'bins': [5, 6, 1, 6, 7, 2], 'goal': 1}},
            {'one': {'bins': [6, 0, 1, 6, 7, 2], 'goal': 2},
             'two': {'bins': [6, 0, 1, 6, 7, 2], 'goal': 2}},
            {'one': {'bins': [6, 0, 1, 7, 8, 0], 'goal': 2},
             'two': {'bins': [6, 0, 1, 6, 7, 2], 'goal': 2}},
            {'on

In [29]:
%%time

rand_vs_rand_p1_results = run_simulation(rand_vs_rand_p1)
rand_vs_rand_p1_results

CPU times: user 10.1 s, sys: 5.88 ms, total: 10.1 s
Wall time: 10.1 s


Counter({<Player.TWO: 1>: 2297, <Player.ONE: 0>: 2392, None: 311})

### With Player.TWO starting

In [30]:
rand_vs_rand_p2 = SimulationLoop(
    player_one=ExampleRandomPlayerStrategy(),
    player_two=ExampleRandomPlayerStrategy(),
    starting_player=Player.TWO
)
pprint_simulation_results(rand_vs_rand_p2)

{'boards': [{'one': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0},
             'two': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0}}],
 'player_strategies': {'one': 'random-selection', 'two': 'random-selection'},
 'starting_player': 'two',
 'turns': [],
 'winning_player': None}


In [31]:
rand_vs_rand_p2.run()
pprint_simulation_results(rand_vs_rand_p2)

{'boards': [{'one': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0},
             'two': {'bins': [4, 4, 4, 4, 4, 4], 'goal': 0}},
            {'one': {'bins': [4, 4, 4, 4, 4, 5], 'goal': 0},
             'two': {'bins': [5, 5, 0, 4, 4, 4], 'goal': 1}},
            {'one': {'bins': [5, 5, 5, 0, 4, 5], 'goal': 1},
             'two': {'bins': [5, 5, 0, 4, 4, 4], 'goal': 1}},
            {'one': {'bins': [6, 0, 5, 0, 4, 5], 'goal': 2},
             'two': {'bins': [5, 5, 0, 5, 5, 5], 'goal': 1}},
            {'one': {'bins': [6, 0, 5, 0, 4, 5], 'goal': 2},
             'two': {'bins': [6, 6, 1, 6, 0, 5], 'goal': 2}},
            {'one': {'bins': [6, 0, 5, 0, 4, 5], 'goal': 2},
             'two': {'bins': [7, 7, 2, 7, 1, 0], 'goal': 2}},
            {'one': {'bins': [7, 1, 6, 1, 0, 5], 'goal': 2},
             'two': {'bins': [7, 7, 2, 7, 1, 0], 'goal': 2}},
            {'one': {'bins': [7, 1, 6, 1, 0, 5], 'goal': 2},
             'two': {'bins': [7, 7, 2, 8, 0, 0], 'goal': 2}},
            {'on

In [32]:
%%time

rand_vs_rand_p2_results = run_simulation(rand_vs_rand_p2)
rand_vs_rand_p2_results

CPU times: user 10.2 s, sys: 36.1 ms, total: 10.2 s
Wall time: 10.2 s


Counter({<Player.TWO: 1>: 2384, <Player.ONE: 0>: 2306, None: 310})